In [ ]:
import os
import warnings
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configuration
RANDOM_STATE = 42
SAMPLE_SIZE = 200_000
TARGET_COLUMN = 'Label'

# Paths
PROJECT_ROOT = Path('..').resolve()
DATA_PATH = PROJECT_ROOT / 'data' / 'processed_randomforest' / 'train.csv'
RESULTS_DIR = PROJECT_ROOT / 'experiments' / 'results'
MODELS_DIR = PROJECT_ROOT / 'models'

# Ensure output directories exist
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Data Path: {DATA_PATH}")
print(f"📊 Results Dir: {RESULTS_DIR}")
print(f"🤖 Models Dir: {MODELS_DIR}")
print(f"✅ Configuration loaded!")

In [ ]:
# Load the pre-processed dataset
print("⏳ Loading dataset (this may take a moment for large files)...")
df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset loaded! Shape: {df.shape}")
print(f"   Total samples: {df.shape[0]:,}")
print(f"   Features: {df.shape[1] - 1}")
print(f"   Target: '{TARGET_COLUMN}'")

In [ ]:
# Display original class distribution
print("\n📊 Original Class Distribution:")
print("=" * 50)
class_dist = df[TARGET_COLUMN].value_counts().sort_index()
class_pct = df[TARGET_COLUMN].value_counts(normalize=True).sort_index() * 100

for label, count in class_dist.items():
    pct = class_pct[label]
    print(f"  Class {label}: {count:>10,} samples ({pct:>6.2f}%)")

print(f"\n  Total: {df.shape[0]:,} samples")

In [ ]:
# Strategic Sampling with Stratification
print(f"\n⚡ Creating stratified sample of {SAMPLE_SIZE:,} rows...")

# Separate features and target
X = df.drop(columns=[TARGET_COLUMN])
y = df[TARGET_COLUMN]

# Calculate sample fraction
sample_frac = SAMPLE_SIZE / len(df)

# Stratified sampling using train_test_split
# We take SAMPLE_SIZE as our "train" portion
X_sample, _, y_sample, _ = train_test_split(
    X, y,
    train_size=SAMPLE_SIZE,
    stratify=y,
    random_state=RANDOM_STATE
)

# Combine back into single DataFrame for PyCaret
df_sample = pd.concat([X_sample, y_sample], axis=1).reset_index(drop=True)

print(f"✅ Stratified sample created! Shape: {df_sample.shape}")

In [ ]:
# Verify stratification integrity
print("\n🔍 Sample Class Distribution (Integrity Check):")
print("=" * 60)

sample_dist = df_sample[TARGET_COLUMN].value_counts().sort_index()
sample_pct = df_sample[TARGET_COLUMN].value_counts(normalize=True).sort_index() * 100
original_pct = class_pct

print(f"{'Class':<10} {'Count':>12} {'Sample %':>12} {'Original %':>12} {'Δ':>8}")
print("-" * 60)

for label in sample_dist.index:
    count = sample_dist[label]
    s_pct = sample_pct[label]
    o_pct = original_pct[label]
    delta = s_pct - o_pct
    print(f"{label:<10} {count:>12,} {s_pct:>11.2f}% {o_pct:>11.2f}% {delta:>+7.3f}%")

print("-" * 60)
print(f"{'Total':<10} {df_sample.shape[0]:>12,}")
print("\n✅ Stratification preserved! Class proportions maintained.")

In [ ]:
# Free up memory from original dataset
del df, X, y
import gc
gc.collect()
print("🧹 Memory cleaned: Original dataset removed from memory.")

In [ ]:
from pycaret.classification import (
    setup, compare_models, plot_model, save_model, create_model, tune_model
)

print("🚀 Initializing PyCaret Classification Environment...")
print("   GPU Mode: DISABLED (Intel OpenCL Issue)")
print("   Normalization: ENABLED")
print("   Session ID: 42 (Reproducibility)")
print("=" * 60)

In [ ]:
# Initialize PyCaret setup
clf_setup = setup(
    data=df_sample,
    target=TARGET_COLUMN,
    session_id=RANDOM_STATE,
    n_jobs=-1,              # Use all CPU cores
    use_gpu=False,          # GPU disabled to prevent LightGBM/Intel OpenCL crash
    normalize=True,         # Apply scaling
    log_experiment=False,   # Lightweight - no MLflow logging
    verbose=True,           # Show setup information
    html=False              # Disable HTML output for cleaner logs
)

In [ ]:
# Define models to compare
BATTLE_MODELS = [
    'lightgbm',   # LightGBM - Fast gradient boosting
    'xgboost',    # XGBoost - Extreme gradient boosting
    'catboost',   # CatBoost - Categorical boosting
    'rf',         # Random Forest
    'et',         # Extra Trees
    'dt',         # Decision Tree
    'ada',        # AdaBoost
    'gbc'         # Gradient Boosting Classifier
]

print("⚔️  THE BATTLE BEGINS!")
print("=" * 60)
print("Contenders:")
for i, model in enumerate(BATTLE_MODELS, 1):
    print(f"  {i}. {model.upper()}")
print("\n📏 Metrics: 5-Fold CV | Optimizing for F1 Score")
print("=" * 60)

In [ ]:
# Run the comparison
print("\n🏁 Starting model comparison (this may take several minutes)...\n")

best_model = compare_models(
    include=BATTLE_MODELS,
    fold=5,                 # 5-fold CV for speed
    sort='F1',              # Optimize for F1 Score
    n_select=1,             # Return only the best model
    verbose=True            # Show progress
)

print("\n" + "=" * 60)
print("🏆 CHAMPION SELECTED!")
print(f"   Best Model: {type(best_model).__name__}")
print("=" * 60)

In [ ]:
import matplotlib.pyplot as plt
import shutil

print("📊 Generating visualizations...")
print(f"   Output directory: {RESULTS_DIR}")
print("=" * 60)

In [ ]:
# Helper function to save PyCaret plots
def save_pycaret_plot(model, plot_type, filename):
    """
    Generate and save PyCaret plot.
    
    Args:
        model: Trained PyCaret model
        plot_type: Type of plot (e.g., 'confusion_matrix', 'auc', 'feature')
        filename: Output filename (without extension)
    """
    try:
        # PyCaret's plot_model saves to current directory by default
        plot_model(model, plot=plot_type, save=True)
        
        # Default name pattern used by PyCaret
        default_name = f"{plot_type.replace('_', ' ').title()}.png"
        
        # Alternative patterns PyCaret might use
        possible_names = [
            default_name,
            f"{plot_type}.png",
            f"{plot_type.title()}.png",
            "Confusion Matrix.png",
            "Feature Importance.png",
            "AUC.png"
        ]
        
        # Find and move the generated file
        for name in possible_names:
            if os.path.exists(name):
                target_path = RESULTS_DIR / filename
                shutil.move(name, target_path)
                print(f"  ✅ Saved: {filename}")
                return target_path
        
        print(f"  ⚠️ Could not locate generated plot for {plot_type}")
        return None
        
    except Exception as e:
        print(f"  ❌ Error generating {plot_type}: {str(e)}")
        return None

In [ ]:
# Generate and save Confusion Matrix
print("\n Generating Confusion Matrix...")
save_pycaret_plot(best_model, 'confusion_matrix', 'confusion_matrix.png')

In [ ]:
# Generate and save Feature Importance
print("\n Generating Feature Importance...")
save_pycaret_plot(best_model, 'feature', 'feature_importance.png')

In [ ]:
# Generate and save AUC/ROC Curve
print("\n Generating ROC/AUC Curve...")
save_pycaret_plot(best_model, 'auc', 'auc.png')

In [ ]:
# Display generated visualizations
print("\n Generated Artifacts:")
print("=" * 60)
for f in RESULTS_DIR.glob('*.png'):
    print(f"  📊 {f.name}")

In [ ]:
# Save the champion model
model_path = MODELS_DIR / 'pycaret_champion'

print("\n Saving champion model...")
save_model(best_model, str(model_path))
print(f"\n Model saved to: {model_path}")
print(f"   File: pycaret_champion.pkl")

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           BENCHMARK COMPLETE - SUMMARY                     ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║   Data Pipeline:                                           ║
║     ├─ Source: data/processed_randomforest/train.csv         ║
║     ├─ Original: ~1.3M+ samples                              ║
║     └─ Stratified Sample: 200,000 samples                    ║
║                                                              ║
║    Battle Results:                                          ║
║     ├─ Algorithms Tested: 8                                  ║
║     ├─ Cross-Validation: 5-Fold                              ║
║     ├─ Primary Metric: F1 Score                              ║
║     └─ Champion: {champion_name:<35} ║
║                                                              ║
║   Artifacts Generated:                                     ║
║     ├─ experiments/results/confusion_matrix.png              ║
║     ├─ experiments/results/feature_importance.png            ║
║     ├─ experiments/results/auc.png                           ║
║     └─ models/pycaret_champion.pkl                           ║
║                                                              ║
╠══════════════════════════════════════════════════════════════╣
║   Next Steps:                                               ║
║     1. Review metrics from compare_models() output           ║
║     2. Fine-tune hyperparameters with tune_model()           ║
║     3. Train on full dataset with identified architecture    ║
║     4. Evaluate on held-out test set                         ║
╚══════════════════════════════════════════════════════════════╝
""".format(champion_name=type(best_model).__name__))

In [ ]:
# Display model info
print("\n Champion Model Details:")
print("=" * 60)
print(best_model)